### Merger Rate of Black Holes in a Primordial Black Hole Cluster

###### Code Writed by Tadeo D.
###### Dedicated to the Moon of my life.
**References:** [arXiv:2105.14523](https://arxiv.org/pdf/2105.14523.pdf).

In [1]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import math
import seaborn as sns
from scipy import stats
from astropy import constants as const
import csv
from scipy import constants
from scipy.integrate import odeint
from tqdm import tqdm
from scipy import integrate
from scipy.integrate import quad, dblquad
from astropy.cosmology import Planck18 as cosmo# WMAP9 as cosmo #
import astropy.units as u
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sympy import *
from sympy import simplify
import pandas as pd

In [2]:
H0 = cosmo.H(0)  #cosmo.H(0)  <Quantity 67.66 km / (Mpc s)>
G = const.G
c = const.c
z_r = 5000 #3450
z_eq = 3411
z_0 = 0
z_init = 6.900e9
rho_c0 = (3*(H0**2))/(8*math.pi*G)
rho_eq = cosmo.critical_density(3411)
rho_crit = cosmo.critical_density(0)
Omega_DM = cosmo.Odm(0)
Omega_M = cosmo.Om(0)
Omega_R = 8.4e-5
Omega_lmb = 1-Omega_M - Omega_R

$$\rho_{i}(r) = \rho_{0,i}\left(\frac{r}{r_{0}}\right)^{-1} \left(1 + \frac{r^{2}}{r^{2}_{0}}\right)^{-2}$$

In [3]:
M = 10**5 * u.M_sun#M is the total mass of the cluster
R = 1.*u.parsec #Scale factor that determines the core radius
v_rel = (G*M/R)**(1./2.)#The characteristic velocity of the PBHs
m_i = 1.*u.M_sun
m_j = 1.*u.M_sun
N_i = M/(2*m_i) #is the number of stars of mass m_i
N_j = M/(2*m_j) #is the number of stars of mass m_i
Energy = G*(m_i*N_i)**(2.)/((2*R)*(1*u.M_sun)) #According to the virial theorem, the total energy

print("N_i =\t", N_i)
print("N_j =\t", N_j)
print("M =\t", M)
print("R =\t", R)
print("v_rel =\t", v_rel.to(u.parsec* u.s**(-1)))
print("m_i =\t", m_i)
print("m_j =\t", m_j)
print("E =\t", Energy.to(u.parsec**2.*u.s**(-2.)))

N_i =	 50000.0
N_j =	 50000.0
M =	 100000.0 solMass
R =	 1.0 pc
v_rel =	 6.7209397035645e-13 pc / s
m_i =	 1.0 solMass
m_j =	 1.0 solMass
E =	 5.6463788123687055e-21 pc2 / s2


In [4]:
sigma = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.))
sigma = sigma.to(u.m**(32./7)/u.s**(18./7.))#.to(u.parsec**2)
print(sigma)
#print(sigma)
#sigma.to(u.AU**2)
#sigma.to(u.parsec**2.)

6.19832413121049e+29 m(32/7) / s(18/7)


In [5]:
def f_i(E): ## eq. 19 shapiro
    return (24. * (2.)**(1./2.) * N_i * R**2. * E**(7./2.))/(7*(math.pi**(3.))*(G**5.)*(M**5.))

def rho_i(r):## eq. 20
    return ((3*M)/(4*(math.pi)*(R**(3.))))*((1 + r**(2.)/R**(2.))**(-5./2.))

def phi(r):## eq. 21
    return (G*M/R)*((1 + r**(2.)/R**(2.))**(-1./2.))

def integrant(E):
    return E**(7./2.)*((2*(a - E))**(3./2.))

In [58]:
f_i(Energy)
#PH_i(1.*u.parsec)
#rho_i(1*u.parsec)

<Quantity 9.90204842e+25 kg(3/2) s3 / (m(9/2) pc(3/2) solMass(3/2))>

In [7]:
n = (3.*N_i)/(4*math.pi*(R**(3.)))
print(n)

11936.620731892152 1 / pc3


In [8]:
r = np.linspace(.1, 1., 10)
print("r(pc)\t\t phi(r) \t\t\t\t\t\t v_i(r)")
phi_array=[]
for i in r:
    i=i*u.parsec
    rho_value = rho_i(i)
    phi_value = phi(i).to(u.parsec**(2.)/u.s**2)
    #print(rho_i(i), "\t", phi_value)
    phi_array.append(phi_value.value)
    a = phi_value.value
    I, err = quad(integrant, 0, a)
    constant = ((4.*math.pi*m_i)/rho_value)
    constant2 = (24. * (2.**(1./2.)) * N_i * R**(2.))/(7.*(math.pi**(3.))*(G**5.)*(M**5.))
    const = (constant*constant2).to(u.s**(10)/u.m**(10))
    vel = (I*const*(u.parsec**(12.)/(u.s**(12.)))).to(u.km**(2.)/(u.s**(2.)))
    result = ((7*math.pi*(phi_value**6.))/(2.**(17./2.)))
    vel2 = (result*const).to(u.km**(2.)/(u.s**(2.)))
    print("%.2f" %i.value,"\t",phi_value,"\t",(vel**(1./2.)),"\t", (vel2**(1./2.)))
print("\n")
#print(phi_array)

r(pc)		 phi(r) 						 v_i(r)
0.10 	 4.494685526656458e-25 pc2 / s2 	 10.343563310882228 km / s 	 10.343563984070622 km / s
0.20 	 4.429383960831521e-25 pc2 / s2 	 10.268149557956848 km / s 	 10.268150226237102 km / s
0.30 	 4.326600034242027e-25 pc2 / s2 	 10.14831399245809 km / s 	 10.148314652939108 km / s
0.40 	 4.194024892154189e-25 pc2 / s2 	 9.991622634567255 km / s 	 9.991623284850354 km / s
0.50 	 4.0402197923747064e-25 pc2 / s2 	 9.806702875881479 km / s 	 9.806703514129477 km / s
0.60 	 3.8733839099994897e-25 pc2 / s2 	 9.602090466506885 km / s 	 9.602091091438126 km / s
0.70 	 3.7005550067478666e-25 pc2 / s2 	 9.385425212121893 km / s 	 9.385425822951948 km / s
0.80 	 3.527264880702962e-25 pc2 / s2 	 9.163039494364146 km / s 	 9.163040090720708 km / s
0.90 	 3.357536254982149e-25 pc2 / s2 	 8.93986327383099 km / s 	 8.93986385566261 km / s
1.00 	 3.1940741978991656e-25 pc2 / s2 	 8.719528902274764 km / s 	 8.719529469766401 km / s




In [9]:
def n_i(r):
    return (rho_i(r)/m_i)

def n_j(r2):
    return (rho_i(r2)/m_j)

def vel_j(r):
    constant = ((4.*math.pi*m_j)/rho_i(r))
    constant2 = (24. * (2.**(1./2.)) * N_j * R**(2.))/(7.*(math.pi**(3.))*(G**5.)*(M**5.))
    const = (constant*constant2).to(u.s**(10)/u.m**(10))
    result = ((7*math.pi*(phi(r)**6.))/(2.**(17./2.)))
    return ((result*const).to(u.km**(2.)/(u.s**(2.))))**(1./2.)
#print(n_i(1*u.parsec))

In [10]:
def rho_bar_DM(z):
    return cosmo.Odm(z)*cosmo.critical_density(z)

m_array = np.linspace(.1, 30, 10)
z = np.arange(0, 11, 1)
rho_cluster = (3*M/(4*math.pi*(1*u.parsec)**3.))
for i in z:
    delta_rusa = (rho_cluster/(rho_bar_DM(i)))
    delt_russ_value = delta_rusa.to((u.parsec**3 *u.M_sun)/(u.M_sun*u.parsec**3)) - 1
    print("Redshift=", i)
    print("Mass \t delta \t\t tau_capture")
    for j in m_array:
        m_a = j*u.M_sun
        m_b = m_a
        n_PBH = (delt_russ_value * (rho_bar_DM(i)/(j*u.M_sun))).to(u.parsec**(-3))
        sigma_PBH_capt = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_a+m_b)**(10./7.) * m_a**(2./7.) * m_b**(2./7.))/(c**(10./7.) * v_rel**(18./7.))
        sigma_PBH_capt_value = sigma_PBH_capt.to(u.parsec**(2.))
        tau_PBH_capt = (sigma_PBH_capt_value*n_PBH*v_rel)
        print("%.1f"%j,"\t","{:.1e}".format(delt_russ_value)," \t","{:.2e}".format(tau_PBH_capt.to(1/u.Gyr)))
    print("\n")

Redshift= 0
Mass 	 delta 		 tau_capture
0.1 	 7.2e+11  	 2.62e-07 1 / Gyr
3.4 	 7.2e+11  	 8.95e-06 1 / Gyr
6.7 	 7.2e+11  	 1.76e-05 1 / Gyr
10.1 	 7.2e+11  	 2.63e-05 1 / Gyr
13.4 	 7.2e+11  	 3.50e-05 1 / Gyr
16.7 	 7.2e+11  	 4.37e-05 1 / Gyr
20.0 	 7.2e+11  	 5.24e-05 1 / Gyr
23.4 	 7.2e+11  	 6.11e-05 1 / Gyr
26.7 	 7.2e+11  	 6.98e-05 1 / Gyr
30.0 	 7.2e+11  	 7.85e-05 1 / Gyr


Redshift= 1
Mass 	 delta 		 tau_capture
0.1 	 9.0e+10  	 2.62e-07 1 / Gyr
3.4 	 9.0e+10  	 8.95e-06 1 / Gyr
6.7 	 9.0e+10  	 1.76e-05 1 / Gyr
10.1 	 9.0e+10  	 2.63e-05 1 / Gyr
13.4 	 9.0e+10  	 3.50e-05 1 / Gyr
16.7 	 9.0e+10  	 4.37e-05 1 / Gyr
20.0 	 9.0e+10  	 5.24e-05 1 / Gyr
23.4 	 9.0e+10  	 6.11e-05 1 / Gyr
26.7 	 9.0e+10  	 6.98e-05 1 / Gyr
30.0 	 9.0e+10  	 7.85e-05 1 / Gyr


Redshift= 2
Mass 	 delta 		 tau_capture
0.1 	 2.7e+10  	 2.62e-07 1 / Gyr
3.4 	 2.7e+10  	 8.95e-06 1 / Gyr
6.7 	 2.7e+10  	 1.76e-05 1 / Gyr
10.1 	 2.7e+10  	 2.63e-05 1 / Gyr
13.4 	 2.7e+10  	 3.50e-05 1 / Gyr
16.7 	 2.7

In [11]:
m_PBH_array = np.linspace(.1, 30, 4)
z = np.arange(0, 11, 1)
for i in z:
    delta_rusa = (rho_cluster/(rho_bar_DM(i)))
    delt_russ_value = delta_rusa.to((u.parsec**3 *u.M_sun)/(u.M_sun*u.parsec**3)) - 1
    print("\t\t redshift = ", i)
    print("mass \t  tau_merg \t ")
    for j in m_PBH_array:
        j = j*u.M_sun
        R_pbh = (2*G*j/c**(2.))
        n_PBH = (delt_russ_value * (rho_bar_DM(i)/(j))).to(u.parsec**(-3))
        tau_merger_PBH = n_PBH*R_pbh*(v_rel**2.)*(j)*((8*math.pi**2.*G)/3.)*(3/(2*math.pi*v_rel**(2.)))**(3./2.)
        print("%.2f"%j.value,"\t ",tau_merger_PBH.to(u.yr**-1)," \t\t", cosmo.age(i)*tau_merger_PBH.to(u.Gyr**-1))

		 redshift =  0
mass 	  tau_merg 	 
0.10 	  4.2079986591960573e-19 1 / yr  		 5.801519486534667e-09
10.07 	  4.236051983590697e-17 1 / yr  		 5.840196283111565e-07
20.03 	  8.430023980589436e-17 1 / yr  		 1.1622377371357784e-06
30.00 	  1.262399597758817e-16 1 / yr  		 1.7404558459604e-06
		 redshift =  1
mass 	  tau_merg 	 
0.10 	  4.207998659155191e-19 1 / yr  		 2.4622444760342563e-09
10.07 	  4.236051983549558e-17 1 / yr  		 2.478659439207818e-07
20.03 	  8.430023980507563e-17 1 / yr  		 4.932696433655291e-07
30.00 	  1.2623995977465568e-16 1 / yr  		 7.386733428102767e-07
		 redshift =  2
mass 	  tau_merg 	 
0.10 	  4.207998659044265e-19 1 / yr  		 1.3788897863859477e-09
10.07 	  4.236051983437893e-17 1 / yr  		 1.3880823849618536e-07
20.03 	  8.430023980285346e-17 1 / yr  		 2.7623758720598483e-07
30.00 	  1.2623995977132795e-16 1 / yr  		 4.136669359157842e-07
		 redshift =  3
mass 	  tau_merg 	 
0.10 	  4.2079986588282547e-19 1 / yr  		 9.02091327233178e-10
10.07 	  4.2360519

In [12]:
#sigma_PBH_capt = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.) * v_rel**(18./7.))
#m_PBH = 1*u.M_sun
#rho_bar_DM = cosmo.Odm(0)*cosmo.critical_density(0)
#delta_PBH_loc = critical_density
#n_PBH = cosmo.critical_density(0)/m_PBH #delta_PBH_loc * (rho_bar_DM/m_PBH)
#print(n_PBH)

In [13]:
def GAMMA(r1):
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/((R.value)**(2.)))
    return r1**(2.)*((1.+(a))**(-9./4.))*(1./((1. + b)**(1./4.)) + 1./((1. + a)**(1./4.)))**(3./7.) - abs((1./((1. + b)**(1./4.)) - 1./((1. + a)**(1./4.))))**(3./7.)

In [14]:
#CONSTANTS
#sigma = 2*math.pi*(85.*math.pi/(6.*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j)**(10./7.) * m_i**(2./7.) * m_j**(2./7.))/(c**(10./7.))
alfa = ((4*24*(2**(1/2))*N_i*(R**(2.)))/(math.pi*(G**(5.))*(M**(5.))*(2**(17./2.))))
betha = ((3.*M)/(m_i*4*math.pi*(R**(3.))))
etha = (G*M/R)
zetha = alfa**(-1./2.) * betha**(3./2.) * etha**(-3.) * alfa**(3./14.) * etha**(9./7.) * betha**(-3./14.)
#print(zetha)

In [15]:
kappa = (1-((8*24*(2**(1./2.))*N_i*m_i)/(3*M*2**(17./2.))))*(1*u.M_sun)
r_star_quad = (4.*(kappa**(2.))*(M*R)**(2.))/((m_i*N_i)**(4.)) - R**(2.)
r_star = (abs(r_star_quad))**(1./2.)
print(r_star)

0.99999999755 pc


In [16]:
E_value = phi(r_star) - ((1./2.)*vel_j(r_star)**(2.))
print(E_value.to(u.parsec**2./u.s**2.))

2.794814926585418e-25 pc2 / s2


In [17]:
def GAMMA(r1,r0): #Equation (11)
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/(R**(2.)))
    return (r1**(2.))*((1.+(a))**(-9./4.))*(1./((1. + a)**(1./4.)) + 1./((1. + b)**(1./4.)))**(3./7.) - abs((1./((1. + a)**(1./4.)) - 1./((1. + b)**(1./4.))))**(3./7.)


#print(m_i)
#print(m_j)
print("radius \t merger_rate \t\t\t Number_Mergers at Universe's time \t time to the first Merger\n")
interval = 0.1
#print(interval)
r = np.arange(.1, 1.1, interval)
colm1 = []
colm2 = []
colm3 = []
colm4 = []
for i in r:
    r0 = i*u.parsec
    colm1.append("%.2f"%r0.value)
    I, err = quad(GAMMA, i+interval, 0., args=(r0,))
    nash_rubi = (14*math.pi/3.)*sigma*(n_j(r0)/vel_j(r0))*zetha*u.parsec**(3)
    #I = I*units
    merger = (I*nash_rubi).to((u.yr**(-1)))
    colm2.append(merger.value)
    colm3.append((merger*(cosmo.age(0).to(u.yr))).value)
    colm4.append(((merger**(-1.)).to(u.Gyr)).value)
    print("%.2f"%r0.value,"\t",merger,"\t\t", merger*(cosmo.age(0).to(u.yr)),"\t\t",(merger**(-1.)).to(u.Gyr))#/(cosmo.age(0).to(u.yr)))

radius 	 merger_rate 			 Number_Mergers at Universe's time 	 time to the first Merger

0.10 	 2.8643837238348164e-11 1 / yr 		 0.39490929861454066 		 34.91152360903682 Gyr
0.20 	 5.3459895830209495e-11 1 / yr 		 0.7370454520684854 		 18.70561071005516 Gyr
0.30 	 7.717361598124973e-11 1 / yr 		 1.0639837918748334 		 12.957796356762165 Gyr
0.40 	 9.513657286362732e-11 1 / yr 		 1.311637018097119 		 10.511204785918041 Gyr
0.50 	 1.0620035870436067e-10 1 / yr 		 1.4641721644893089 		 9.416164052550787 Gyr
0.60 	 1.1122344703597124e-10 1 / yr 		 1.533424907179087 		 8.99090998030825 Gyr
0.70 	 1.1169588477501144e-10 1 / yr 		 1.5399383520995753 		 8.952881317107572 Gyr
0.80 	 1.0901668007360483e-10 1 / yr 		 1.5030004641806771 		 9.17290821298933 Gyr
0.90 	 1.0427737527277867e-10 1 / yr 		 1.4376602124804228 		 9.589807927022568 Gyr
1.00 	 9.827557359181852e-11 1 / yr 		 1.3549140610996162 		 10.175468465373072 Gyr


In [18]:
number_col_array = []
prob_array = []
print("Black holes Masses:\t", m_i, m_j)
print("Merger_Rate \t\t\t | Probability \t\t\t | Collision Number \t |")
print("----------------------------------------------------------------------------------")
for i in colm2:
    GAMMA_value = i*(1/u.yr)
    Prob = GAMMA_value*(1e6*u.yr)
    prob_array.append(Prob)
    Num_Col = (N_j+N_i)*Prob #numero de choques
    number_col_array.append(Num_Col)
    print(GAMMA_value, "\t | " , Prob, "\t | " , Num_Col, " \t | ")

Black holes Masses:	 1.0 solMass 1.0 solMass
Merger_Rate 			 | Probability 			 | Collision Number 	 |
----------------------------------------------------------------------------------
2.8643837238348164e-11 1 / yr 	 |  2.8643837238348163e-05 	 |  2.864383723834816  	 | 
5.3459895830209495e-11 1 / yr 	 |  5.3459895830209497e-05 	 |  5.34598958302095  	 | 
7.717361598124973e-11 1 / yr 	 |  7.717361598124972e-05 	 |  7.717361598124972  	 | 
9.513657286362732e-11 1 / yr 	 |  9.513657286362732e-05 	 |  9.513657286362733  	 | 
1.0620035870436067e-10 1 / yr 	 |  0.00010620035870436067 	 |  10.620035870436066  	 | 
1.1122344703597124e-10 1 / yr 	 |  0.00011122344703597124 	 |  11.122344703597124  	 | 
1.1169588477501144e-10 1 / yr 	 |  0.00011169588477501144 	 |  11.169588477501144  	 | 
1.0901668007360483e-10 1 / yr 	 |  0.00010901668007360483 	 |  10.901668007360483  	 | 
1.0427737527277867e-10 1 / yr 	 |  0.00010427737527277866 	 |  10.427737527277866  	 | 
9.827557359181852e-11 1 / yr 	 |

In [30]:
t_k = 1e6
t_k_1 = 1e5
delta_time = np.arange(0.0, t_k, t_k_1)
for i in colm2:
    GAMMA_value = i*(1/u.yr)
    delta_Prob = GAMMA_value*((t_k)*u.yr) - GAMMA_value*(t_k_1*u.yr)
    dn = delta_Prob/((t_k)*u.yr - t_k_1*u.yr)
    Num_Col = (N_i+N_j)*delta_Prob
    print(GAMMA_value,"\t ",delta_Prob,"\t\t ",dn)

2.8643837238348164e-11 1 / yr 	  2.5779453514513346e-05 		  2.8643837238348164e-11 1 / yr
5.3459895830209495e-11 1 / yr 	  4.8113906247188544e-05 		  5.3459895830209495e-11 1 / yr
7.717361598124973e-11 1 / yr 	  6.945625438312475e-05 		  7.717361598124973e-11 1 / yr
9.513657286362732e-11 1 / yr 	  8.562291557726459e-05 		  9.513657286362732e-11 1 / yr
1.0620035870436067e-10 1 / yr 	  9.558032283392461e-05 		  1.0620035870436067e-10 1 / yr
1.1122344703597124e-10 1 / yr 	  0.00010010110233237412 		  1.1122344703597125e-10 1 / yr
1.1169588477501144e-10 1 / yr 	  0.0001005262962975103 		  1.1169588477501144e-10 1 / yr
1.0901668007360483e-10 1 / yr 	  9.811501206624434e-05 		  1.0901668007360483e-10 1 / yr
1.0427737527277867e-10 1 / yr 	  9.384963774550079e-05 		  1.0427737527277866e-10 1 / yr
9.827557359181852e-11 1 / yr 	  8.844801623263667e-05 		  9.827557359181852e-11 1 / yr


In [20]:
#f = open("MR_T1.txt", "w")
#f.write("Mpbh = 1 u.Solar Mass")
#f.write("\n")
#f.write("radius \t merger_rate \t Number_Mergers at Universe's time \t time to the first Merger")
#f.write("\n")
#for i in range(len(r)):
#    print(colm1[i],"\t ",colm2[i],"\t ",colm3[i],"\t ",colm4[i],"\t ",prob_array[i],"\t ", number_col_array[i],file=f)
#f.close()
#read_file = pd.read_csv (r"MR_T1.txt")
#read_file.to_csv (r"MR_T1.csv", index=None)

## SECOND GENERATION

In [35]:
Ni_2_values = []
Nj_2_values = []
print("PBHs con 2 M_sun \t PBHs con 1 M_sun")# \t  Number_pbhs at cluster")
for i in number_col_array:
    number_binarys = round(float(i))
    N_j_2 = number_binarys #PBHs de 2 masas solares
    N_i_2 = N_i + (N_j - 2*round(float(i))) #PBHs de 1 masa solar
    Ni_2_values.append(N_i_2)
    Nj_2_values.append(N_j_2)
    #Nt = (N_i + N_j - N_j_2) #Numero de agujeros negros totales en el cluster
    print(N_j_2,"\t\t\t", N_i_2)

PBHs con 2 M_sun 	 PBHs con 1 M_sun
3 			 99994.0
5 			 99990.0
8 			 99984.0
10 			 99980.0
11 			 99978.0
11 			 99978.0
11 			 99978.0
11 			 99978.0
10 			 99980.0
10 			 99980.0


In [95]:
#Datos de entrada para la segunda generación
M = 10**5 * u.M_sun #M is the total mass of the cluster
R = 1.*u.parsec #Scale factor that determines the core radius
m_j_2 = 2.*m_j #masa segunda geenración especie j
v_rel = (G*M/R)**(1./2.)#The characteristic velocity of the PBHs
#Energy = G*(m_i*N_i)**(2.)/((2*R)*(1*u.M_sun)) #According to the virial theorem, the total energy
Energy_mi = G*(m_i*N_i)**(2.)/((2*R)*(1*u.M_sun)) #According to the virial theorem, the total energy
Energy_mj2 = G*(m_j_2*N_i)**(2.)/((2*R)*(1*u.M_sun)) #According to the virial theorem, the total energy
print("M =\t", M)
print("R =\t", R)
print("v_rel =\t", v_rel.to(u.parsec* u.s**(-1)))
print("m_i_2 =\t", m_i)
print("m_j_2 =\t", m_j_2)
#print("E =\t", Energy.to(u.parsec**2.*u.s**(-2.)))
print("Energy to m_i = \t", Energy_mi.to(u.parsec**2.*u.s**(-2.)))
print("Energy to m_j_2 = \t", Energy_mj2.to(u.parsec**2.*u.s**(-2.)))
print("\n")

M =	 100000.0 solMass
R =	 1.0 pc
v_rel =	 6.7209397035645e-13 pc / s
m_i_2 =	 1.0 solMass
m_j_2 =	 2.0 solMass
Energy to m_i = 	 5.6463788123687055e-21 pc2 / s2
Energy to m_j_2 = 	 2.2585515249474822e-20 pc2 / s2




In [42]:
print("M_i \t\t\t M_j")# \t\t\t  vel_rel_i \t\t\t  vel_rel_j")
Mi_array = []
Mj_array = []
v_ri_array = []
v_rj_array = []
for i,j in zip(Ni_2_values, Nj_2_values):
    Mi = i*u.M_sun
    Mj = j*2*u.M_sun
    #v_ri = (G*Mi/R)**(1./2.)#The characteristic velocity of the PBHs QUITAR ESTO
    #v_rj = (G*Mj/R)**(1./2.)#The characteristic velocity of the PBHs QUITAR ESTO
    Mi_array.append(Mi)
    Mj_array.append(Mj)
    v_ri_array.append(v_ri)
    v_rj_array.append(v_rj)
    print(Mi,"\t",Mj,"\t ")#, v_ri.to(u.km/u.s)," \t ",v_rj.to(u.km/u.s))

M_i 			 M_j
99994.0 solMass 	 6.0 solMass 	 
99990.0 solMass 	 10.0 solMass 	 
99984.0 solMass 	 16.0 solMass 	 
99980.0 solMass 	 20.0 solMass 	 
99978.0 solMass 	 22.0 solMass 	 
99978.0 solMass 	 22.0 solMass 	 
99978.0 solMass 	 22.0 solMass 	 
99978.0 solMass 	 22.0 solMass 	 
99980.0 solMass 	 20.0 solMass 	 
99980.0 solMass 	 20.0 solMass 	 


In [47]:
sigma_2 = 2*math.pi*(85*math.pi/(6*math.sqrt(2)))**(2./7.) * (G**(2.) * (m_i+m_j_2)**(10./7.) * m_i**(2./7.) * m_j_2**(2./7.))/(c**(10./7.))
sigma_2 = sigma.to(u.parsec**(32./7)/u.yr**(18./7.))#.to(u.parsec**2)
#print(m_i, m_j_2)
print(sigma_2)

4.963594304558011e-27 pc(32/7) / yr(18/7)


In [24]:
def f_i2(E, Ni_2): ## eq. 19 shapiro
    return (24. * (2.)**(1./2.) * Ni_2 * R**2. * E**(7./2.))/(7*(math.pi**(3.))*(G**5.)*(M**5.))

def f_j2(E, Nj_2):
    return (24. * (2.)**(1./2.) * Nj_2 * R**2. * E**(7./2.))/(7*(math.pi**(3.))*(G**5.)*(M**5.))

def rho_i2(r, M_i):## eq. 20
    return ((3*M_i)/(4*(math.pi)*(R**(3.))))*((1 + r**(2.)/R**(2.))**(-5./2.))

def rho_j2(r, M_j):## eq. 20
    return ((3*M_j)/(4*(math.pi)*(R**(3.))))*((1 + r**(2.)/R**(2.))**(-5./2.))

#def integrant(E):
#    return E**(7./2.)*((2*(a - E))**(3./2.))

In [78]:
r = np.linspace(.1, 1., 10)
print("N_i \t\t f_i2 \t\t\t\t\t\t N_j \t\t f_j2")
for h,i,j in zip (r, Ni_2_values, Nj_2_values):
    h = h*u.parsec
    print(i,"\t",f_i2(Energy_mi, i).to(u.s**3./(u.m**6.))," \t\t ",j,"\t",f_j2(Energy_mj2, j).to(u.s**3./(u.m**6.)))
print("\n")
print("M_i \t\t\t rho_i2 \t\t\t\t M_j \t\t\t  rho_j2")
for h,k,l in zip (r, Mi_array, Mj_array):
    h = h*u.parsec
    print(k,"\t",rho_i2(h,k),"\t",l,"\t",rho_j2(h,l))

N_i 		 f_i2 						 N_j 		 f_j2
99994.0 	 1.3029996518219577e-44 s3 / m6  		  3 	 5.003818892129844e-47 s3 / m6
99990.0 	 1.3029475287084978e-44 s3 / m6  		  5 	 8.339698153549741e-47 s3 / m6
99984.0 	 1.3028693440383084e-44 s3 / m6  		  8 	 1.3343517045679588e-46 s3 / m6
99980.0 	 1.3028172209248487e-44 s3 / m6  		  10 	 1.6679396307099482e-46 s3 / m6
99978.0 	 1.302791159368119e-44 s3 / m6  		  11 	 1.834733593780943e-46 s3 / m6
99978.0 	 1.302791159368119e-44 s3 / m6  		  11 	 1.834733593780943e-46 s3 / m6
99978.0 	 1.302791159368119e-44 s3 / m6  		  11 	 1.834733593780943e-46 s3 / m6
99978.0 	 1.302791159368119e-44 s3 / m6  		  11 	 1.834733593780943e-46 s3 / m6
99980.0 	 1.3028172209248487e-44 s3 / m6  		  10 	 1.6679396307099482e-46 s3 / m6
99980.0 	 1.3028172209248487e-44 s3 / m6  		  10 	 1.6679396307099482e-46 s3 / m6


M_i 			 rho_i2 				 M_j 			  rho_j2
99994.0 solMass 	 23285.303226685715 solMass / pc3 	 6.0 solMass 	 1.3972020257226863 solMass / pc3
99990.0 solMass 	 21641.

In [79]:
def n_i_2(r, M_i):
    return (rho_i2(r, M_i)/m_i)

def n_j_2(r, M_j):
    return (rho_j2(r, M_j)/m_j_2)

In [86]:
alfa_2 = ((4*24*(2**(1/2))*(R**(2.)))/(math.pi*(G**(5.))*(M**(5.))*(2**(17./2.))))

def vel_i2(r, N_i_2, M_i):
    return (alfa_2**(1./2.))*(N_i_2**(1./2.))*((phi(r)**(3))/(n_j_2(r, M_i)**(1./2.)))

def vel_j2(r, N_j_2, M_j):
    return (alfa_2**(1./2.))*(N_j_2**(1./2.))*((phi(r)**(3))/(n_j_2(r, M_j)**(1./2.)))


for h,i,j,k,l in zip(r, Ni_2_values, Nj_2_values, Mi_array, Mj_array):
    h = h*u.parsec
    print(vel_i2(h,i,k).to(u.km/u.s), "\t", vel_j2(h,j,l).to(u.km/u.s))

#print("\n")
#for h in r:
    #h = h*u.parsec
    #print("r = ","%.2f"%h.value)
    #print("vel_rel = ")
    #for i,j,k,l in zip(Ni_2_values, Nj_2_values, Mi_array, Mj_array):
    #    print(vel_i2(h,i,k).to(u.km/u.s), "\t", vel_j2(h,j,l).to(u.km/u.s))

20.687127968141255 km / s 	 14.628008469546563 km / s
20.536300452474208 km / s 	 14.521357310428877 km / s
20.296629305878216 km / s 	 14.351884217416098 km / s
19.98324656970071 km / s 	 14.130289159558187 km / s
19.61340702825896 km / s 	 13.868773111853802 km / s
19.204182182876256 km / s 	 13.57940744865368 km / s
18.770851645903896 km / s 	 13.272996487465313 km / s
18.326080181441423 km / s 	 12.958495568865626 km / s
17.879727711325224 km / s 	 12.642876710447096 km / s
17.439058939532803 km / s 	 12.331276833655531 km / s


In [87]:
#CONSTANTS SECOND GENERATION
betha_2 = (3./(4*math.pi*(R**(3.))))
etha_2 = (G*M/R)
#zetha2 = alfa**(-1./2.) * betha**(3./2.) * etha**(-3.) * alfa**(3./14.) * etha**(9./7.) * betha**(-3./14.)
#print(zetha)

In [93]:
def GAMMA_2(r1,r0): #Equation (11)
    a=(r1**(2.)/((R.value)**(2.)))
    b=(r0**(2.)/(R**(2.)))
    return (r1**(2.))*((1.+(a))**(-9./4.))*(1./((1. + a)**(1./4.)) + 1./((1. + b)**(1./4.)))**(3./7.) - abs((1./((1. + a)**(1./4.)) - 1./((1. + b)**(1./4.))))**(3./7.)

print("radius \t merger_rate \t\t\t Number_Mergers at Universe's time \t time to the first Merger\n")

for h,i,j,k in zip(r, Ni_2_values, Nj_2_values, Mj_array) :
    r0 = h*u.parsec
    I, err = quad(GAMMA_2, h+interval, 0., args=(r0,))
    nash_rubi = (14*math.pi/3.)*sigma_2*(i)*(n_j_2(r0, k)/vel_j2(r0, j, k))*(alfa_2**(-2./7.))*(betha_2**(9./7.))*(etha_2**(-12./7.))
    #I = I*units
    merger2 = (I*nash_rubi*(u.parsec**(3.))).to(u.yr**(-1))
    #print("%.2f"%r0.value,"\t",merger*u.parsec**(3))
    print("%.2f"%r0.value,"\t",merger2,"\t\t", merger2*(cosmo.age(0).to(u.yr)),"\t\t",(merger2**(-1.)).to(u.Gyr))#/(cosmo.age(0).to(u.yr)))

radius 	 merger_rate 			 Number_Mergers at Universe's time 	 time to the first Merger

0.10 	 4.984284521873743e-16 1 / yr 		 6.871775901565559e-06 		 2006306.0116481269 Gyr
0.20 	 1.550354852351873e-15 1 / yr 		 2.137456452678947e-05 		 645013.6228380295 Gyr
0.30 	 3.580682188613248e-15 1 / yr 		 4.9366454637359946e-05 		 279276.3912921541 Gyr
0.40 	 5.5174329865443984e-15 1 / yr 		 7.606821574701249e-05 		 181243.70562157134 Gyr
0.50 	 6.774847837953328e-15 1 / yr 		 9.340405008043099e-05 		 147604.7911213455 Gyr
0.60 	 7.0952861070743824e-15 1 / yr 		 9.782189574317749e-05 		 140938.64361620965 Gyr
0.70 	 7.12542436492918e-15 1 / yr 		 9.823740844742398e-05 		 140342.51839398188 Gyr
0.80 	 6.954509648630642e-15 1 / yr 		 9.58810268573905e-05 		 143791.58999325024 Gyr
0.90 	 6.0475526158065335e-15 1 / yr 		 8.337691427199346e-05 		 165356.14711086475 Gyr
1.00 	 5.69947891947216e-15 1 / yr 		 7.85780621439849e-05 		 175454.63613937044 Gyr


In [ ]:
f = open("MR_T0.1.txt", "w")
f.write("Mpbh = 0.1 u.Solar Mass")
f.write("\n")
f.write("radius \t merger_rate \t Number_Mergers at Universe's time \t time to the first Merger")
f.write("\n")
for i in range(len(r)):
    print(colm1[i],"\t ",colm2[i],"\t ",colm3[i],"\t ",colm4[i], file=f)
f.close()

In [ ]:
read_file = pd.read_csv (r"MR_T0.1.txt")
read_file.to_csv (r"MR_T0.1.csv", index=None)